In [122]:
import pandas as pd

# Mapowania

In [123]:
mapowanie_pkd_2007_2025 = pd.read_excel(r"..\data\mapowanie_pkd.xlsx", sheet_name="MAP_PKD_2007_2025")
# mapowanie_pkd_2025_2007 = pd.read_excel(r"..\data\mapowanie_pkd.xlsx", sheet_name="MAP_PKD_2025_2007")
pkd_2025 = pd.read_excel(r"..\data\mapowanie_pkd.xlsx", sheet_name="PKD_2025")
# pkd_2007 = pd.read_excel(r"..\data\mapowanie_pkd.xlsx", sheet_name="PKD_2007")

In [124]:
mapowanie_pkd_2007_2025_with_og = pd.concat([
    mapowanie_pkd_2007_2025,
    pd.DataFrame({'symbol_2007': ['OG'], 'symbol_2025': ['OG']})
], ignore_index=True)

In [125]:
pkd_2025

,typ,symbol,nazwa
0,SEKCJA,A,"ROLNICTWO, LEŚNICTWO I RYBACTWO"
1,DZIAŁ,01,"UPRAWY ROLNE, CHÓW I HODOWLA ZWIERZĄT, ŁOWIECT..."
2,GRUPA,01.1,Uprawy rolne inne niż wieloletnie
3,KLASA,01.11,"Uprawa zbóż innych niż ryż, roślin strączkowyc..."
4,PODKLASA,01.11.Z,"Uprawa zbóż innych niż ryż, roślin strączkowyc..."
...,...,...,...
1770,SEKCJA,V,ORGANIZACJE I ZESPOŁY EKSTERYTORIALNE
1771,DZIAŁ,99,ORGANIZACJE I ZESPOŁY EKSTERYTORIALNE
1772,GRUPA,99.0,Organizacje i zespoły eksterytorialne
1773,KLASA,99.00,Organizacje i zespoły eksterytorialne


In [126]:
pkd_2025_with_og = pd.concat([
    pkd_2025,
    pd.DataFrame({'typ': ['OGÓŁEM'], 'symbol': ['OG'], 'nazwa': ['OGÓŁEM']})
], ignore_index=True)
pkd_2025_with_og

,typ,symbol,nazwa
0,SEKCJA,A,"ROLNICTWO, LEŚNICTWO I RYBACTWO"
1,DZIAŁ,01,"UPRAWY ROLNE, CHÓW I HODOWLA ZWIERZĄT, ŁOWIECT..."
2,GRUPA,01.1,Uprawy rolne inne niż wieloletnie
3,KLASA,01.11,"Uprawa zbóż innych niż ryż, roślin strączkowyc..."
4,PODKLASA,01.11.Z,"Uprawa zbóż innych niż ryż, roślin strączkowyc..."
...,...,...,...
1771,DZIAŁ,99,ORGANIZACJE I ZESPOŁY EKSTERYTORIALNE
1772,GRUPA,99.0,Organizacje i zespoły eksterytorialne
1773,KLASA,99.00,Organizacje i zespoły eksterytorialne
1774,PODKLASA,99.00.Z,Organizacje i zespoły eksterytorialne


# Upadlosci - KRZ_PKD
krz_pkd.csv - przypisanie liczby upadłości firm z Krajowego Rejestru Zadłużonych do PKD
(2007) od 2018 roku -
**poziom podklasy**

In [127]:
upadlosci = pd.read_csv(r"..\data\krz_pkd.csv", sep=";", encoding="utf-8")

In [128]:
upadlosci["pkd_formatted"] = (
    upadlosci["pkd"]
    .str.strip()
    .str.upper()
    .str.replace(r"^(\d{2})(\d{2})([A-Z])$", r"\1.\2.\3", regex=True)
)

In [129]:
upadlosci["pkd_formatted_no_letter"] = upadlosci["pkd_formatted"].str.replace(r"\.[A-Z]$", "", regex=True)
upadlosci["pkd_letter"] = upadlosci["pkd_formatted"].str.extract(r"\.([A-Z])$")

upadlosci_mapped = upadlosci.merge(
    mapowanie_pkd_2007_2025[["symbol_2007", "symbol_2025"]],
    how="left",
    left_on="pkd_formatted_no_letter",
    right_on="symbol_2007"
)
unmapped_mask = upadlosci_mapped["symbol_2025"].isna()
if unmapped_mask.any():
    missing_codes = (
        upadlosci_mapped.loc[unmapped_mask, "pkd_formatted_no_letter"]
        .dropna()
        .unique()
        .tolist()
    )
    raise ValueError(f"Niezmapowane kody PKD 2007: {missing_codes}")


ValueError: Niezmapowane kody PKD 2007: ['25.20', '49.61', '13.12', '47.49', '68.39', '68.19', 'ZIELON']

In [130]:
upadlosci_mapped["pkd_2025"] = upadlosci_mapped["symbol_2025"] + '.' + upadlosci_mapped["pkd_letter"]

In [131]:
upadlosci_final = upadlosci_mapped[["rok", "pkd_2025", "liczba_upadlosci"]]

In [132]:
upadlosci_final

,rok,pkd_2025,liczba_upadlosci
0,2018,01.11.Z,6
1,2018,01.13.Z,4
2,2018,01.19.Z,2
3,2018,01.24.Z,1
4,2018,01.29.Z,1
...,...,...,...
2540,2024,96.21.Z,12
2541,2024,96.22.Z,12
2542,2024,96.30.Z,1
2543,2024,93.13.Z,1


# Wskaznik finansowy
wszystkie poziomy oprócz podklas

In [133]:
wskaznik_finansowy = pd.read_csv(r"..\data\wsk_fin.csv", sep=";", encoding="utf-8")

In [134]:
#  OG - ogółem; SEK_A - sekctor; 
# wskaznik_finansowy

In [135]:
wskaznik_finansowy["PKD_formatted"] = wskaznik_finansowy["PKD"].str.replace(r"^SEK_", "", regex=True).str.rstrip('.')

In [136]:
wskaznik_finansowy = wskaznik_finansowy.drop(columns=["NAZWA_PKD", "NUMER_NAZWA_PKD","PKD"])

In [137]:
wskaznik_finansowy_mapped = wskaznik_finansowy.join(
    mapowanie_pkd_2007_2025_with_og[["symbol_2007", "symbol_2025"]].set_index("symbol_2007"),
    on="PKD_formatted"
)
unmapped_mask = wskaznik_finansowy_mapped["symbol_2025"].isna()
if unmapped_mask.any():
    missing_codes = (
        wskaznik_finansowy_mapped.loc[unmapped_mask, "PKD_formatted"]
        .dropna()
        .unique()
        .tolist()
    )
    raise ValueError(f"Niezmapowane kody PKD 2007: {missing_codes}")

In [138]:
wskaznik_finansowy_mapped = wskaznik_finansowy_mapped.drop(columns=["PKD_formatted"])

In [139]:
wskaznik_finansowy_transposed = wskaznik_finansowy_mapped.melt(
    id_vars=['symbol_2025', 'WSKAZNIK'],
    var_name='rok',
    value_name='wartosc'
)
wskaznik_finansowy_transposed['rok'] = wskaznik_finansowy_transposed['rok'].astype(int)

In [140]:
wskaznik_finansowy_transposed['wartosc'] = wskaznik_finansowy_transposed['wartosc'].replace('bd', pd.NA)

In [141]:
wskaznik_finansowy_final = wskaznik_finansowy_transposed.rename(columns={'symbol_2025': 'pkd_2025'})

In [142]:
wskaznik_finansowy_final


,pkd_2025,WSKAZNIK,rok,wartosc
0,OG,EN Liczba jednostek gospodarczych,2005,"46 396,00"
1,OG,PEN Liczba rentownych jednostek gospodarczych,2005,"35 860,00"
2,OG,GS Przychody ogółem,2005,"1 679 774,30"
3,OG,PNPM Przychody netto,2005,<NA>
4,OG,GS (I) Przychody netto ze sprzedaży i zrównane...,2005,"1 614 314,24"
...,...,...,...,...
418555,99.0,STL Zobowiązania krótkoterminowe,2024,<NA>
418556,99.0,LTC Długoterminowe kredyty bankowe,2024,<NA>
418557,99.0,STC Krótkoterminowe kredyty bankowe,2024,<NA>
418558,99.0,INV Zapasy,2024,<NA>


In [143]:
upadlosci_final['WSKAZNIK'] = 'Upadłość'
upadlosci_final = upadlosci_final.rename(columns={'liczba_upadlosci': 'wartosc'})

C:\Users\tusiu\AppData\Local\Temp\ipykernel_17660\2794861416.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  upadlosci_final['WSKAZNIK'] = 'Upadłość'


In [144]:
upadlosci_final

,rok,pkd_2025,wartosc,WSKAZNIK
0,2018,01.11.Z,6,Upadłość
1,2018,01.13.Z,4,Upadłość
2,2018,01.19.Z,2,Upadłość
3,2018,01.24.Z,1,Upadłość
4,2018,01.29.Z,1,Upadłość
...,...,...,...,...
2540,2024,96.21.Z,12,Upadłość
2541,2024,96.22.Z,12,Upadłość
2542,2024,96.30.Z,1,Upadłość
2543,2024,93.13.Z,1,Upadłość


In [145]:
combined_data = pd.concat([wskaznik_finansowy_final, upadlosci_final], ignore_index=True)
combined_data

,pkd_2025,WSKAZNIK,rok,wartosc
0,OG,EN Liczba jednostek gospodarczych,2005,"46 396,00"
1,OG,PEN Liczba rentownych jednostek gospodarczych,2005,"35 860,00"
2,OG,GS Przychody ogółem,2005,"1 679 774,30"
3,OG,PNPM Przychody netto,2005,<NA>
4,OG,GS (I) Przychody netto ze sprzedaży i zrównane...,2005,"1 614 314,24"
...,...,...,...,...
421100,96.21.Z,Upadłość,2024,12
421101,96.22.Z,Upadłość,2024,12
421102,96.30.Z,Upadłość,2024,1
421103,93.13.Z,Upadłość,2024,1
